In [ ]:
from flask import Flask, request, jsonify
import time
import re
import json
import requests
import google.genai as genai

client = genai.Client(api_key="your api key!!")

app = Flask(__name__)

def extract_json_from_text(text):
    start_idx = text.find('{')
    end_idx = text.rfind('}')
    if start_idx != -1 and end_idx != -1 and end_idx > start_idx:
        return text[start_idx:end_idx+1]
    else:
        return None

@app.route('/interview_analyze', methods=['POST'])
def interview_analyze():
    data = request.json
    que = data.get('question')
    ans = data.get('answer')
    timeline = data.get('timeline')
    if not que or not ans:
        return jsonify({"error": "Missing 'question' or 'answer' in JSON body."}), 400

    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=f"너는 개발자 면접 전문 코치야. [{que}]라는 질문에 모의 면접자가 [{ans}]라고 답변을 했어. 이 답변에대해 답변의 긍정적 부분, 부정적 부분, 개선된 답변, 종합적으로 평가했을때 답변의 점수를 100점 만점으로 평가해줘. 답변형식은 [positive: ~~, negative: ~~, improved_answer: ~~, score: ~~] 이런식으로 json 형식으로 답변해줘. positive와 negative는 한두줄정도로 짧게 대답해줘"
    )
    
    print(response.text)

    # 문자열에서 JSON 부분 추출
    json_str = extract_json_from_text(response.text)
    
    if json_str:
        ans_json = json.loads(json_str)  # 문자열 -> 딕셔너리 변환
    else:
        ans_json = {}
    
    ans_json["original_answer"] = ans
    ans_json["timeline_answer"] = timeline
    return jsonify(ans_json)

@app.route('/generate-questions', methods=['POST'])
def generate_questions():
    data = request.json
    text = data.get('resume_text')

    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=f"너는 개발자 면접 전문 코치야. 면접자가 제출한 자소서가 [{text}] 이거야. 이 이력서를 기반으로 면접자에게 할 질문 3개 생성해줘. 각 질문 시작에 ###으로 구분자 넣어줘. 다른 답변하지말고 질문 3개만 만들어줘. 각 질문이 너무 길지 않게 해줘. 1. 2. 이런거 없이 구분자와 답변으로만 작성해줘"
    )

    print(response.text)

    que_str = response.text

    questions = [q.strip() for q in que_str.split('###') if q.strip()]

    return jsonify({"questions": questions})


@app.route('/overall', methods=['POST'])
def overall():
    data = request.json
    vision = data.get('vision')
    timeline = data.get('timeline')
    emotion = data.get('emotion')
    if not timeline:
        return jsonify({"error": "Missing 'timeline' in JSON body."}), 400
    if not vision:
        return jsonify({"error": "Missing 'vision' in JSON body."}), 400
    if not emotion:
        return jsonify({"error": "Missing 'emotion' in JSON body."}), 400

    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=f"면접자의 질문 답변을 감정, 집중도, 답변을 종합적으로 분석해줘. 1초는 30프레임이야. timeline은 start가 답변 시작시간이고 end가 답변 종료시간이야. 이를 종합적으로 분석해서 두세줄정도로 종합분석 결과를 알려줘. 데이터는 다음과 같아 vision[{vision}], timeline[{timeline}], emotion[{emotion}]"
    )

    print(response.text)
    overall = response.text
    return jsonify({"overall": overall})


@app.route('/overoverall', methods=['POST'])
def overoverall():
    # 1. 입력 JSON 파싱 (리스트여야 함)
    data_list = request.get_json()
    if not data_list or not isinstance(data_list, list):
        return jsonify({"error": "Input JSON 데이터가 리스트가 아닙니다."}), 400

    # 2. overall만 추출해서 한글 텍스트로 정리
    overalls = [item.get('overall') for item in data_list if item.get('overall')]
    if not overalls:
        return jsonify({"error": "overall 필드가 없습니다."}), 400

    # 3. 여러 overall을 한 개의 문자열로 합치기 (분석 프롬프트용)
    overall_texts = "\n\n".join(overalls)


    # 5. LLM 모델에 분석 요청
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=f"[{overall_texts}]이건 이 면접자가 본 변접 평가를 합쳐둔 데이터야 이데이터를 보고 이 면접자의 전반적 평가를 두세줄로 요약해서 설명해줘. 다른 말이나 소제목 달지말고 요약만 작성해줘."
    )
    overall = response.text
    print(overall)

    return jsonify({"overall": overall})

@app.route('/overall_compare', methods=['POST'])
def overall_compare():
    data = request.json
    previous = data.get("previous_interview")
    current = data.get("current_interview")

    if not previous or not current:
        return jsonify({"error": "Missing previous or current interview data"}), 400

    prompt = (
        f"이전 인터뷰의 평균 감정 {previous['emotion_avg']}, 평균 집중도 {previous['vision_avg']}, "
        f"평균 점수 {previous['score_avg']}와, 현재 인터뷰의 평균 감정 {current['emotion_avg']}, "
        f"평균 집중도 {current['vision_avg']}, 평균 점수 {current['score_avg']}를 종합적으로 비교해 분석 결과를 2-3문장으로 작성해줘."
    )

    # LLM API 호출 부분
    response = client.models.generate_content(model="gemini-2.0-flash", contents=prompt)

    # 예시 응답 (실제 응답에서는 response.text 등)
    response_text = response.text

    print(response_text)

    return jsonify({"overall_compare": response_text})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)